In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, LineString
import pyvista as pv
import flopy
from flopy.utils.gridgen import Gridgen

ws = './ex-pyvista'
name = 'mymodel'
gridgen_ws = pl.Path(ws, "gridgen")
gridgen_ws.mkdir(parents=True, exist_ok=True)

In [ ]:
Lx = 100.0
Ly = 100.0
nlay = 3
nrow = 5
ncol = 5
dx = Lx / ncol
dy = Ly / nrow
delr = np.ones(ncol, dtype=float) * Lx / ncol
delc = np.ones(nrow, dtype=float) * Ly / nrow
xx, yy = np.meshgrid(
    np.arange(dx / 2, Lx + dx / 2, dx), 
    np.arange(dy / 2, Ly + dy / 2, dy),
)
r = np.sqrt((xx - 50) ** 2 + (yy - 50) ** 2)
top = (r.max() - r) / 5 + 5
botm = np.zeros((nlay, nrow, ncol), dtype=float)
botm[0, :, :] = -5.0
botm[1, :, :] = -7.0
botm[2, :, :] = -10.0 - xx / 5.
sg = flopy.discretization.StructuredGrid(
    delr=delr, 
    delc=delc,
    top=top,
    botm=botm,
)

In [ ]:
g = Gridgen(sg, model_ws=gridgen_ws, surface_interpolation="interpolate")
g.add_refinement_features([(50., 50.), (50., 0.)], "point", level=2, layers=[0])
g.build()
#g.plot(layer=1)
usg_props = g.get_gridprops_unstructuredgrid()
usg = flopy.discretization.UnstructuredGrid(**usg_props)

In [ ]:
vtk_export = flopy.export.vtk.Vtk(modelgrid=usg)
vtk_export.add_array(np.arange(usg.nnodes), name="node")
pvgrid = vtk_export.to_pyvista()
pvgrid.cell_data['node-number'] = range(pvgrid.n_cells)
pvgrid.plot(show_edges=True)